In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchmetrics
!pip install segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.4 MB/s eta 0:00:00


In [3]:
import os
import sys
sys.path.append('..')

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import warnings
from torch.utils.data import DataLoader
from pathlib import Path
warnings.filterwarnings("ignore")

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 환경에 따라 PROJECT_ROOT 설정
if IN_COLAB:
    PROJECT_ROOT = '/content/drive/Othercomputers/내 Mac/Road_Lane_segmentation'
else:
    PROJECT_ROOT = Path.cwd().parent

# sys.path 추가 (import용)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# 작업 디렉토리 변경 (파일 접근용)
os.chdir(PROJECT_ROOT)

print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"Current working directory: {os.getcwd()}")

from src.utils import setup_workspace, load_config, get_device, seed
from src.data import DatasetLoader, get_transforms, split_dataset
from src.models import DeepLabV3Plus
from src.models.losses import get_loss
from src.inference import Inferencer, visualize_prediction, visualize_comparison

Environment: Colab
PROJECT_ROOT: /content/drive/Othercomputers/내 Mac/Road_Lane_segmentation
Current working directory: /content/drive/Othercomputers/내 Mac/Road_Lane_segmentation


In [4]:
!nvidia-smi

Tue Jan 20 00:50:00 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             55W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
SEED = 42
DATA_ROOT = "dataset/Multi Camera Semantic Segmentation"
CONFIG_ROOT = "configs/config.yaml"

setup_workspace(PROJECT_ROOT)
seed(SEED)
get_device()

Env: Colab | Root: /content/drive/Othercomputers/내 Mac/Road_Lane_segmentation
Device: NVIDIA A100-SXM4-80GB


device(type='cuda')

In [6]:
config = load_config(CONFIG_ROOT)
config

{'data': {'root': 'dataset/Multi_Camera_Semantic_Segmentation',
  'num_classes': 62,
  'img_size': [768, 576]},
 'model': {'name': 'deeplabv3_resnet101', 'pretrained': True},
 'training': {'batch_size': 16,
  'accumulation_steps': 2,
  'epochs': 80,
  'lr': 0.001,
  'weight_decay': 0.0001,
  'dropout': 0.1,
  'early_stop': 10,
  'num_workers': 4},
 'checkpoint': {'dir': 'checkpoints', 'exp_name': 'exp1'},
 'loss': {'type': 'ce+dice',
  'weights': {'ce': 0.5, 'dice': 0.5},
  'ignore_index': 255},
 'wandb': {'enabled': True,
  'project': 'Road_Lane_Segmentation',
  'run_name': 'baseline',
  'tags': ['deeplabv3', 'segmentation']}}

In [7]:
from scripts.train import Trainer

trainer = Trainer("configs/config.yaml")
trainer.train()

INFO:Trainer:Gradient accumulation enabled with 2 steps.


Device: NVIDIA A100-SXM4-80GB
Train samples: 515
Val samples: 182
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth


100%|██████████| 233M/233M [00:05<00:00, 48.6MB/s]
INFO:Trainer:Loss: ce+dice
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: moonstalker9010 (moonstalker9010-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Validating: 100%|██████████| 12/12 [02:29<00:00, 12.47s/it]
INFO:Trainer:Epoch 1/80 | Train Loss: 1.0863 | Val Loss: 0.6766 | mIoU: 0.0742 | Accuracy: 0.1065 | Dice: 0.0882
INFO:Trainer:Logging validation images to W&B for epoch 1...
INFO:Trainer:Saved: checkpoints/deeplabv3_resnet101_exp1/best.pt
INFO:Trainer:mIoU improved. Early stopping counter reset.
Validating: 100%|██████████| 12/12 [00:08<00:00,  1.46it/s]
INFO:Trainer:Epoch 2/80 | Train Loss: 0.7020 | Val Loss: 0.6689 | mIoU: 0.0965 | Accuracy: 0.1299 | Dice: 0.1184
INFO:Trainer:Saved: checkpoints/deeplabv3_resnet101_exp1/best.pt
INFO:Trainer:mIoU improved. Early stopping counter reset.
Validating: 100%|██████████| 12/12 [00:08<00:00,  1.44it/s]
INFO:Trainer:Epoch 3/80 | Train Loss: 0.6227 | Val Loss: 0.3925 | mIoU: 0.1383 | Accuracy: 0.1809 | Dice: 0.1647
INFO:Trainer:Saved: checkpoints/deeplabv3_resnet101_exp1/best.pt
INFO:Trainer:mIoU improved. Early stopping counter reset.
Validating: 100%|██████████| 12/12 [00:08<00:00,  1

accuracy,▁▁▂▂▂▄▄▄▄▄▅▆▆▅▆▆▆▆▆▇▆▆▆▇▇▇▇█▇▇█▇▇██▆▄▆██
dice,▁▂▂▂▂▄▃▄▄▅▅▅▆▆▆▇▆▇▆▅▃▆▇▇▇▇██████▆▃▆▇▇▇██
miou,▁▂▃▄▄▄▄▄▅▆▅▆▆▇▆▆▅▆▃▆▆▇▇████████▇▇█▆▇▇▇██
train_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁
val_loss,▆▆▃▃█▂▂▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▅▂▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.5368
best_miou,0.42979
dice,0.53146
early_stopped,True
miou,0.41996
stopped_epoch,73


INFO:Trainer:Training complete. Best mIoU: 0.4298


In [8]:
from scripts.evaluate import Evaluator

checkpoint_path = "checkpoints/deeplabv3_resnet101_exp1/best.pt"
evaluator = Evaluator(config_path=CONFIG_ROOT, checkpoint_path=checkpoint_path)
metrics = evaluator.evaluate()
metrics

Device: NVIDIA A100-SXM4-80GB


INFO:Evaluator:Loaded 182 samples from val split
INFO:Evaluator:Number of classes: 62


Val samples: 182
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:03<00:00, 52.0MB/s]
INFO:Evaluator:Model: deeplabv3_resnet101
INFO:Evaluator:Checkpoint: checkpoints/deeplabv3_resnet101_exp1/best.pt
Evaluating: 100%|██████████| 12/12 [00:09<00:00,  1.29it/s]
INFO:Evaluator:Evaluation Results | mIoU: 0.4298 | Accuracy: 0.5227 | Dice: 0.5365


{'miou': 0.42978569865226746,
 'accuracy': 0.5227274894714355,
 'dice': 0.5365280508995056}

In [9]:
import pandas as pd

metrics_df = pd.DataFrame([metrics])
metrics_df

,miou,accuracy,dice
0,0.429786,0.522727,0.536528


In [10]:
import numpy as np

def create_colormap(num_classes):
    colormap = np.random.randint(0, 255, size=(num_classes, 3), dtype=np.uint8)
    colormap[0] = [0, 0, 0]  # background는 검정
    return colormap

def mask_to_color(mask, colormap):
    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for class_id, color in enumerate(colormap):
        color_mask[mask == class_id] = color
    return color_mask

In [11]:
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

def visualize_segmentation(
    model,
    dataloader,
    device,
    num_classes,
    num_samples=3,
    alpha=0.5
):
    """모델 추론 결과를 시각화"""
    model.eval()
    colormap = create_colormap(num_classes)

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)["out"]
            preds = outputs.argmax(dim=1)

            images = images.cpu()
            masks = masks.cpu()
            preds = preds.cpu()

            for i in range(min(num_samples, images.size(0))):
                img_tensor = images[i].clone()

                # 역정규화 (De-normalization)
                for c in range(3):
                    img_tensor[c] = img_tensor[c] * STD[c] + MEAN[c]

                # Tensor → Numpy Image
                img_np = img_tensor.permute(1, 2, 0).numpy()
                img_np = np.clip(img_np, 0, 1)

                gt_mask = masks[i].numpy()
                pred_mask = preds[i].numpy()

                gt_color = mask_to_color(gt_mask, colormap)
                pred_color = mask_to_color(pred_mask, colormap)

                overlay = img_np * (1 - alpha) + (pred_color / 255.0) * alpha

                fig, axes = plt.subplots(1, 4, figsize=(24, 6))

                axes[0].imshow(img_np)
                axes[0].set_title("Image")

                axes[1].imshow(gt_color)
                axes[1].set_title("GT Mask")

                axes[2].imshow(pred_color)
                axes[2].set_title("Pred Mask")

                axes[3].imshow(overlay)
                axes[3].set_title("Overlay (Prediction)")

                for ax in axes:
                    ax.axis("off")

                plt.tight_layout()
                plt.show()

            break  # 첫 batch만 시각화

In [12]:
visualize_segmentation(
    model=evaluator.model,
    dataloader=evaluator.dataloader,
    device=evaluator.device,
    num_classes=evaluator.num_classes,
    num_samples=3
)

Output hidden; open in https://colab.research.google.com to view.